In [157]:
import numpy as np
import os
import cv2

In [158]:
def compute_errors(gt, pred):
    """ Evaluates the predicted depth data on ground truth

    Args:
        gt: numpy array (2D) of the ground depth map
        pred: numpy array (2D) of the predicted depth

    Returns:
        -abs_rel
        -sq_rel
        -rmse
        -rmse_log
        -a1
        -a2x
        -a3
    """


    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25   ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()

    rmse = (gt - pred) ** 2
    rmse = np.sqrt(rmse.mean())

    rmse_log = (np.log(gt) - np.log(pred)) ** 2
    rmse_log = np.sqrt(rmse_log.mean())

    abs_rel = np.mean(np.abs(gt - pred) / gt)

    sq_rel = np.mean(((gt - pred)**2) / gt)

    return abs_rel, sq_rel, rmse, rmse_log, a1, a2, a3

In [183]:
def read_file_data(files, data_root):
    """ Reads in the files for evaluation of ground truth data

        Args:
            files (list string): a list of filenames (two files per line left/right)
            data_root (string): root folder 

        Returns:
            -gt_files (list string): filenames for the velodyne points (gt from laser range scanner)
            -gt_calib (list string): path for respective camera calibration files of the particular sequence 
            -im_sizes (list tuples): sizes of each gt file for later resizing 
            -im_files (list string): list of filenames for left images
            -cams (list int): list of camera that image was taken from (2 is left and 3 is right)
        """

    gt_files = []
    gt_calib = []
    im_sizes = []
    im_files = []
    cams = []
    num_probs = 0

    for filename in files:
        filename = filename.split()[0]
        splits = filename.split('/')
        camera_id = np.int32(splits[2][-1:])  # 2 is left, 3 is right
        date = splits[0]
        im_id = splits[4][:10]
        file_root = '{}/{}'

        im = filename
        vel = '{}/{}/velodyne_points/data/{}.bin'.format(splits[0], splits[1], im_id)

        if os.path.isfile(data_root + im):
            gt_files.append(data_root + vel)
            gt_calib.append(data_root + date + '/')
            im_sizes.append(cv2.imread(data_root + im).shape[:2])
            im_files.append(data_root + im)
            cams.append(2)
        else:
            num_probs += 1
            print('{} missing'.format(data_root + im))
            
    print (num_probs, 'files missing')

    return gt_files, gt_calib, im_sizes, im_files, cams

In [184]:
files = open('test_files.txt', 'r')
gt_files, gt_calib, im_sizes, im_files, cams = read_file_data(files,'../../data/kitti/')

0 files missing


In [199]:
def load_velodyne_points(file_name):
    """ Reads in the ground truth depth file
        # adapted from https://github.com/hunse/kitti

        Args:
            file_name(string): path to velodyne bin file to be loaded 

        Returns:
            -points (2D): points 
        """
    
    points = np.fromfile(file_name, dtype=np.float32).reshape(-1, 4)
    points[:, 3] = 1.0  # homogeneous
    return points

def lin_interp(shape, xyd):
    """ Reads in the ground truth depth file
        # adapted from https://github.com/hunse/kitti

        Args:
            shape(tuple): path to velodyne bin file to be loaded 

        Returns:
            -points (2D): points 
    """
    
    # taken from https://github.com/hunse/kitti
    m, n = shape
    ij, d = xyd[:, 1::-1], xyd[:, 2]
    f = LinearNDInterpolator(ij, d, fill_value=0)
    J, I = np.meshgrid(np.arange(n), np.arange(m))
    IJ = np.vstack([I.flatten(), J.flatten()]).T
    disparity = f(IJ).reshape(shape)
    return disparity

def read_calib_file(path):
    """ Reads in in the calibration files for the camera and puts them into a dictionary
    check here for contens of the calibration file:  
    https://github.com/yanii/kitti-pcl/blob/master/KITTI_README.TXT
    adapted from https://github.com/hunse/kitti

    Args:
        -path(string): path to the calibration file

    Returns:
        -data (dict): dictionary that is storing the different calibration arrays 
    """
    # taken from https://github.com/hunse/kitti
    float_chars = set("0123456789.e+- ")
    data = {}
    with open(path, 'r') as f:
        for line in f.readlines():
            key, value = line.split(':', 1)
            value = value.strip()
            data[key] = value
            if float_chars.issuperset(value):
                # try to cast to float array
                try:
                    data[key] = np.array(map(float, value.split(' ')))
                except ValueError:
                    # casting error: data[key] already eq. value, so pass
                    pass

    return data



def get_focal_length_baseline(calib_dir, cam):
    """ Reads in the ground truth depth file
        # adapted from https://github.com/hunse/kitti

        Args:
            -calib_dir(string): path to camera calibration folder 
            -cam (int) camera that was used (either camera 1 or camera 2)

        Returns:
            -focal length (float): focal length of the camera used 
            -baseline (float): length of the baseline of the camera used 
        """
    cam2cam = read_calib_file(calib_dir + 'calib_cam_to_cam.txt')
    P2_rect = cam2cam['P_rect_02'].reshape(3,4)
    P3_rect = cam2cam['P_rect_03'].reshape(3,4)

    # cam 2 is left of camera 0  -6cm
    # cam 3 is to the right  +54cm
    b2 = P2_rect[0,3] / -P2_rect[0,0]
    b3 = P3_rect[0,3] / -P3_rect[0,0]
    baseline = b3-b2

    if cam==2:
        focal_length = P2_rect[0,0]
    elif cam==3:
        focal_length = P3_rect[0,0]

    return focal_length, baseline


def sub2ind(matrixSize, rowSub, colSub):
    """ Converts an index into our number so can be acessed in an array 
    
    Args:
        -matrixSize (tuple): size of the matrix
        -rowSub (int): row index
        -colSub (int): column index
        
    Returns:
        -index (int): particular index in a flat array
    """
        
    m, n = matrixSize
    return rowSub * (n-1) + colSub - 1

(121151, 4)